<a href="https://colab.research.google.com/github/chennatiru/nlp/blob/master/baseline/Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!ls

sample_data


In [0]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!ls

drive  sample_data


In [0]:
%cd drive/

/content/drive


In [0]:
!ls
%cd 'My Drive'

'My Drive'  'Shared drives'
/content/drive/My Drive


In [0]:
%cd commonsenseqa_baseline_1

/content/drive/My Drive/commonsenseqa_baseline_1


In [0]:
import os
import sys
import json
import datetime
import pprint
import tensorflow as tf

In [0]:
import os
import sys
import json
import datetime
import pprint
import tensorflow as tf

# If you want to use TPU, first switch to tpu runtime in colab
USE_TPU = False #@param{type:"boolean"}

# We will use base uncased bert model, you can give try with large models
# For large model TPU is necessary
BERT_MODEL = 'uncased_L-12_H-768_A-12' #@param {type:"string"}

# BERT checkpoint bucket
BERT_PRETRAINED_DIR = 'gs://cloud-tpu-checkpoints/bert/' + BERT_MODEL
print('***** BERT pretrained directory: {} *****'.format(BERT_PRETRAINED_DIR))
!gsutil ls $BERT_PRETRAINED_DIR

OUTPUT_DIR = 'output_directory'
os.mkdir(OUTPUT_DIR)

print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

if USE_TPU:
  # getting info on TPU runtime
  assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; Change notebook runtype to TPU'
  TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  print('TPU address is', TPU_ADDRESS)


***** BERT pretrained directory: gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12 *****
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/bert_config.json
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/bert_model.ckpt.index
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/bert_model.ckpt.meta
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/checkpoint
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/vocab.txt
***** Model output directory: output_directory *****


In [0]:
# Clone BERT repo and add bert in system path
!test -d bert || git clone -q https://github.com/google-research/bert.git
if not 'bert' in sys.path:
  sys.path += ['bert']
  
#Directories
TASK_DATA_DIR = '' 

In [0]:
!git clone https://github.com/chennatiru/qadataset.git

Cloning into 'qadataset'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 18 (delta 8), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (18/18), done.


In [0]:
with open('qadataset/dev_qtoken_split.jsonl') as json_file:
  lines=[]
  for line in json_file.read().split('\n'):
    ekpiece = json.loads(line)
    # print(ekpiece["question"]["stem"])
    answer_key = ekpiece["answerKey"]
    for a in ekpiece["question"]["choices"]:
      if(a["label"]==answer_key):
        right_ans = a["text"]
        lines.append([ekpiece["question"]["stem"],right_ans])
  print(lines)

[['What is it called when you slowly cook using a grill?', 'barbeque'], ['The drug kingpin told his man to run errands, this was code to go to all the dealers to do what they had?', 'take money'], ["Though he could've kept going his body appreciated the rest, it had been constantly what during the day?", 'moving'], ['To prevent any glare during the big football game he made sure to clean the dust of his what?', 'television'], ['What kind of driving leads to accidents?', 'dangerous'], ['What might be the result of a season of successful skiing?', 'healthy body'], ['Jim enjoyed killing people, but he did it too often and the cops found out.  What might the cops do to him?', 'get arrested'], ['What do drugs do?', 'cause accidents'], ['Reading newspaper one of many ways to practice your what?', 'literacy'], ['It was a great rest, she had never felt this much what in the morning?', 'wakefulness'], ['How do geese normally get from place to place?', 'fly'], ['A soccer field that is not contai

In [0]:
import modeling
import optimization
import tokenization
import run_classifier

# Model Hyper Parameters
TRAIN_BATCH_SIZE = 16 # For GPU, reduce to 16
EVAL_BATCH_SIZE = 4
PREDICT_BATCH_SIZE = 4
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 2.0
WARMUP_PROPORTION = 0.1
MAX_SEQ_LENGTH = 128 

#Model configs
SAVE_CHECKPOINTS_STEPS = 1000
ITERATIONS_PER_LOOP = 1000
NUM_TPU_CORES = 8
VOCAB_FILE = os.path.join(BERT_PRETRAINED_DIR, 'vocab.txt')
CONFIG_FILE = os.path.join(BERT_PRETRAINED_DIR, 'bert_config.json')
INIT_CHECKPOINT = os.path.join(BERT_PRETRAINED_DIR, 'bert_model.ckpt')
DO_LOWER_CASE = BERT_MODEL.startswith('uncased')

In [0]:
class CSClass(run_classifier.DataProcessor):
  

  def get_train_examples(self, data_dir):
    """Reading train_qtoken_split.jsonl and converting to list of InputExample"""
    
    with open('qadataset/train_qtoken_split.jsonl') as json_file:
      lines=[]
      for line in json_file.read().split('\n'):
        ekpiece = json.loads(line)
        # print(ekpiece["question"]["stem"])
        answer_key = ekpiece["answerKey"]
        for a in ekpiece["question"]["choices"]:
          if(a["label"]==answer_key):
            right_ans = a["text"]
            lines.append([ekpiece["question"]["stem"], right_ans, 1])
          else:
            wrong_ans = a['text']
            lines.append([ekpiece['question']['stem'], wrong_ans, 0])

    return self._create_examples(lines, 'train')

  def get_dev_examples(self, data_dir):
    """Reading dev_qtoken_split.jsonl and converting to list of InputExample"""
    
    with open('qadataset/dev_qtoken_split.jsonl') as json_file:
      lines=[]
      for line in json_file.read().split('\n'):
        ekpiece = json.loads(line)
        # print(ekpiece["question"]["stem"])
        answer_key = ekpiece["answerKey"]
        for a in ekpiece["question"]["choices"]:
          if(a["label"]==answer_key):
            right_ans = a["text"]
            lines.append([ekpiece["question"]["stem"],right_ans, 1])
          else:
            wrong_ans = a['text']
            lines.append([ekpiece['question']['stem'], wrong_ans, 0])
            
    return self._create_examples(lines, 'dev')
  
#   def get_test_examples(self, data_dir):
#     """Reading train.tsv and converting to list of InputExample"""
    
#     with open('qadataset/train_qtoken_split.jsonl') as json_file:
#       lines=[]
#       for line in json_file.read().split('\n'):
#         ekpiece = json.loads(line)
#         # print(ekpiece["question"]["stem"])
#         answer_key = ekpiece["answerKey"]
#         for a in ekpiece["question"]["choices"]:
#           ans = a["text"]
#           lines.append([ekpiece["question"]["stem"], ans])
            
#     return self._create_examples(lines, 'test')
  
#   def get_predict_examples(self, sentence_pairs):
#     """Given question pairs, conevrting to list of InputExample"""
#     examples = []
#     for (i, qpair) in enumerate(sentence_pairs):
#       guid = "predict-%d" % (i)
#       # converting questions to utf-8 and creating InputExamples
#       text_a = tokenization.convert_to_unicode(qpair[0])
#       text_b = tokenization.convert_to_unicode(qpair[1])
#       # We will add label  as 0, because None is not supported in converting to features
#       examples.append(
#           run_classifier.InputExample(guid=guid, text_a=text_a, text_b=text_b, label=0))
#     return examples
  
  def _create_examples(self, lines, set_type):
    """Creates examples for the training, dev and test sets."""
    examples = []
    for i in range(len(lines)):
      guid = "%s-%d" % (set_type, i)
      
      if set_type=='test':
        text_a = tokenization.convert_to_unicode(lines[i][1])
        text_b = tokenization.convert_to_unicode(line[2])
        label = 0 # We will use zero for test as convert_example_to_features doesn't support None
        
      else:
        text_a = tokenization.convert_to_unicode(lines[i][0])
        text_b = tokenization.convert_to_unicode(lines[i][1])
        label = int(lines[i][2])
      examples.append(
          run_classifier.InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
    return examples

  def get_labels(self):
    "return class labels"
    return [0,1]

In [0]:
# Instantiate an instance of QQPProcessor and tokenizer
processor = CSClass()
label_list = processor.get_labels()
tokenizer = tokenization.FullTokenizer(vocab_file=VOCAB_FILE, do_lower_case=DO_LOWER_CASE)

In [0]:
# Converting training examples to features
print("################  Processing Training Data #####################")
TRAIN_TF_RECORD = os.path.join(OUTPUT_DIR, "train.tf_record")
train_examples = processor.get_train_examples(TASK_DATA_DIR)
num_train_examples = len(train_examples)
num_train_steps = int( num_train_examples / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)
run_classifier.file_based_convert_examples_to_features(train_examples, label_list, MAX_SEQ_LENGTH, tokenizer, TRAIN_TF_RECORD)

################  Processing Training Data #####################


INFO:tensorflow:Writing example 0 of 48110
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: train-0
INFO:tensorflow:tokens: [CLS] the sanctions against the school were a punish ##ing blow , and they seemed to what the efforts the school had made to change ? [SEP] enforce [SEP]
INFO:tensorflow:input_ids: 101 1996 17147 2114 1996 2082 2020 1037 16385 2075 6271 1010 1998 2027 2790 2000 2054 1996 4073 1996 2082 2018 2081 2000 2689 1029 102 16306 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [0]:
# CREATING CLASSIFICATION MODEL

def create_model(bert_config, is_training, input_ids, input_mask, segment_ids,
                 labels, num_labels, use_one_hot_embeddings):
  """Creates a classification model."""
  # Bert Model instant 
  model = modeling.BertModel(
      config=bert_config,
      is_training=is_training,
      input_ids=input_ids,
      input_mask=input_mask,
      token_type_ids=segment_ids,
      use_one_hot_embeddings=use_one_hot_embeddings)

  # Getting output for last layer of BERT
  output_layer = model.get_pooled_output()
  
  # Number of outputs for last layer
  hidden_size = output_layer.shape[-1].value
  
  # We will use one layer on top of BERT pretrained for creating classification model
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):
    if is_training:
      # 0.1 dropout
      output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)
    
    # Calcaulte prediction probabilites and loss
    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    probabilities = tf.nn.softmax(logits, axis=-1)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)

    return (loss, per_example_loss, logits, probabilities)

In [0]:
def model_fn_builder(bert_config, num_labels, init_checkpoint, learning_rate,
                     num_train_steps, num_warmup_steps, use_tpu,
                     use_one_hot_embeddings):
  """Returns `model_fn` closure for TPUEstimator."""

  def model_fn(features, labels, mode, params):  
    """The `model_fn` for TPUEstimator."""

    # reading features input
    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]
    is_real_example = None
    if "is_real_example" in features:
      is_real_example = tf.cast(features["is_real_example"], dtype=tf.float32)
    else:
      is_real_example = tf.ones(tf.shape(label_ids), dtype=tf.float32)
    
    # checking if training mode
    is_training = (mode == tf.estimator.ModeKeys.TRAIN)
    
    # create simple classification model
    (total_loss, per_example_loss, logits, probabilities) = create_model(
        bert_config, is_training, input_ids, input_mask, segment_ids, label_ids,
        num_labels, use_one_hot_embeddings)
    
    # getting variables for intialization and using pretrained init checkpoint
    tvars = tf.trainable_variables()
    initialized_variable_names = {}
    scaffold_fn = None
    if init_checkpoint:
      (assignment_map, initialized_variable_names
      ) = modeling.get_assignment_map_from_checkpoint(tvars, init_checkpoint)
      if use_tpu:

        def tpu_scaffold():
          tf.train.init_from_checkpoint(init_checkpoint, assignment_map)
          return tf.train.Scaffold()

        scaffold_fn = tpu_scaffold
      else:
        tf.train.init_from_checkpoint(init_checkpoint, assignment_map)

    output_spec = None
    if mode == tf.estimator.ModeKeys.TRAIN:
      # defining optimizar function
      train_op = optimization.create_optimizer(
          total_loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu)
      
      # Training estimator spec
      output_spec = tf.contrib.tpu.TPUEstimatorSpec(
          mode=mode,
          loss=total_loss,
          train_op=train_op,
          scaffold_fn=scaffold_fn)
    elif mode == tf.estimator.ModeKeys.EVAL:
      # accuracy, loss, auc, F1, precision and recall metrics for evaluation
      def metric_fn(per_example_loss, label_ids, logits, is_real_example):
        predictions = tf.argmax(logits, axis=-1, output_type=tf.int32)
        loss = tf.metrics.mean(values=per_example_loss, weights=is_real_example)
        accuracy = tf.metrics.accuracy(
            labels=label_ids, predictions=predictions, weights=is_real_example)
        f1_score = tf.contrib.metrics.f1_score(
            label_ids,
            predictions)
        auc = tf.metrics.auc(
            label_ids,
            predictions)
        recall = tf.metrics.recall(
            label_ids,
            predictions)
        precision = tf.metrics.precision(
            label_ids,
            predictions) 
        return {
            "eval_accuracy": accuracy,
            "eval_loss": loss,
            "f1_score": f1_score,
            "auc": auc,
            "precision": precision,
            "recall": recall
        }

      eval_metrics = (metric_fn,
                      [per_example_loss, label_ids, logits, is_real_example])
      # estimator spec for evalaution
      output_spec = tf.contrib.tpu.TPUEstimatorSpec(
          mode=mode,
          loss=total_loss,
          eval_metrics=eval_metrics,
          scaffold_fn=scaffold_fn)
    else:
      # estimator spec for predictions
      output_spec = tf.contrib.tpu.TPUEstimatorSpec(
          mode=mode,
          predictions={"probabilities": probabilities},
          scaffold_fn=scaffold_fn)
    return output_spec

  return model_fn

In [0]:
# Define TPU configs
if USE_TPU:
  tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)
else:
  tpu_cluster_resolver = None
run_config = tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    model_dir=OUTPUT_DIR,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=ITERATIONS_PER_LOOP,
        num_shards=NUM_TPU_CORES,
        per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [0]:
# create model function for estimator using model function builder
model_fn = model_fn_builder(
    bert_config=modeling.BertConfig.from_json_file(CONFIG_FILE),
    num_labels=len(label_list),
    init_checkpoint=INIT_CHECKPOINT,
    learning_rate=LEARNING_RATE,
    num_train_steps=num_train_steps,
    num_warmup_steps=num_warmup_steps,
    use_tpu=USE_TPU,
    use_one_hot_embeddings=True)

In [0]:
# Defining TPU Estimator
estimator = tf.contrib.tpu.TPUEstimator(
    use_tpu=USE_TPU,
    model_fn=model_fn,
    config=run_config,
    train_batch_size=TRAIN_BATCH_SIZE,
    eval_batch_size=EVAL_BATCH_SIZE,
    predict_batch_size=PREDICT_BATCH_SIZE)

INFO:tensorflow:Using config: {'_model_dir': 'output_directory', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7feeeb5d7ba8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=1000, num_shards=8, num_cores_per_replica=None, per_host_input_for_training=3, tpu_job_

In [0]:
# Train the model.
print('QQP on BERT base model normally takes about 1 hour on TPU and 15-20 hours on GPU. Please wait...')
print('***** Started training at {} *****'.format(datetime.datetime.now()))
print('  Num examples = {}'.format(num_train_examples))
print('  Batch size = {}'.format(TRAIN_BATCH_SIZE))
tf.logging.info("  Num steps = %d", num_train_steps)
# we are using `file_based_input_fn_builder` for creating input function from TF_RECORD file
train_input_fn = run_classifier.file_based_input_fn_builder(TRAIN_TF_RECORD,
                                                            seq_length=MAX_SEQ_LENGTH,
                                                            is_training=True,
                                                            drop_remainder=True)
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print('***** Finished training at {} *****'.format(datetime.datetime.now()))

QQP on BERT base model normally takes about 1 hour on TPU and 15-20 hours on GPU. Please wait...
***** Started training at 2019-09-24 05:51:03.381381 *****
  Num examples = 48110
  Batch size = 16
INFO:tensorflow:  Num steps = 6013

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
Use `tf.data.experimental.map_and_batch(...)`.
Instructions for updating:
Use `tf.data.Dataset.map(map_func, num_parallel_calls)` followed by `tf.data.Dataset.batch(batch_size, drop_remainder)`. Static tf.data optimizations will take care of using the fused implementation.

Instructions for updating:
Use `tf.cast` instead.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Running train on CPU



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.dense instead.


Instructions f

In [0]:
# eval the model on train set.
print('***** Started Train Set evaluation at {} *****'.format(datetime.datetime.now()))
print('  Num examples = {}'.format(num_train_examples))
print('  Batch size = {}'.format(EVAL_BATCH_SIZE))
# eval input function for train set
train_eval_input_fn = run_classifier.file_based_input_fn_builder(TRAIN_TF_RECORD,
                                                           seq_length=MAX_SEQ_LENGTH,
                                                           is_training=False,
                                                           drop_remainder=True)
# evalute on train set
result = estimator.evaluate(input_fn=train_eval_input_fn, 
                            steps=int(num_train_examples/EVAL_BATCH_SIZE))
print('***** Finished evaluation at {} *****'.format(datetime.datetime.now()))
print("***** Eval results *****")
for key in sorted(result.keys()):
  print('  {} = {}'.format(key, str(result[key])))

***** Started Train Set evaluation at 2019-09-24 07:35:16.693092 *****
  Num examples = 48110
  Batch size = 4
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Running eval on CPU
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-09-24T07:35:22Z
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from output_directory/model.ckpt-6013
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [1202/12027]
INFO:tensorflow:Evaluation [2404/12027]
INFO:tensorflow:Evaluation [3606/12027]
INFO:tensorflow:Evaluation [4808/12027]
INFO:tensorflow:Evaluation [6010/12027]
INFO:tensorflow:Evaluation [7212/12027]
INFO:tensorflow:Evaluation [8414/12027]
INFO:tensorflow:Evaluation [9616/12027]
INFO:tensorflow:Evaluation [10818/12027]
INFO:tensorflow:Evaluation [12020/12027]
INFO:tensorflow:Evaluation [12027/12

In [0]:
# Converting eval examples to features
print("################  Processing Dev Data #####################")
EVAL_TF_RECORD = os.path.join(OUTPUT_DIR, "eval.tf_record")
eval_examples = processor.get_dev_examples(TASK_DATA_DIR)
num_eval_examples = len(eval_examples)
run_classifier.file_based_convert_examples_to_features(eval_examples, label_list, MAX_SEQ_LENGTH, tokenizer, EVAL_TF_RECORD)

################  Processing Dev Data #####################
INFO:tensorflow:Writing example 0 of 6135
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: dev-0
INFO:tensorflow:tokens: [CLS] what is it called when you slowly cook using a grill ? [SEP] neighbor ' s house [SEP]
INFO:tensorflow:input_ids: 101 2054 2003 2009 2170 2043 2017 3254 5660 2478 1037 18651 1029 102 11429 1005 1055 2160 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [0]:
# Eval the model on Dev set.
print('***** Started Dev Set evaluation at {} *****'.format(datetime.datetime.now()))
print('  Num examples = {}'.format(num_eval_examples))
print('  Batch size = {}'.format(EVAL_BATCH_SIZE))

# eval input function for dev set
eval_input_fn = run_classifier.file_based_input_fn_builder(EVAL_TF_RECORD,
                                                           seq_length=MAX_SEQ_LENGTH,
                                                           is_training=False,
                                                           drop_remainder=True)
# evalute on dev set
result = estimator.evaluate(input_fn=eval_input_fn, steps=int(num_eval_examples/EVAL_BATCH_SIZE))
print('***** Finished evaluation at {} *****'.format(datetime.datetime.now()))
print("***** Eval results *****")
for key in sorted(result.keys()):
  print('  {} = {}'.format(key, str(result[key])))

***** Started Dev Set evaluation at 2019-09-24 07:53:31.881383 *****
  Num examples = 6135
  Batch size = 4
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Running eval on CPU
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-09-24T07:53:37Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from output_directory/model.ckpt-6013
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [153/1533]
INFO:tensorflow:Evaluation [306/1533]
INFO:tensorflow:Evaluation [459/1533]
INFO:tensorflow:Evaluation [612/1533]
INFO:tensorflow:Evaluation [765/1533]
INFO:tensorflow:Evaluation [918/1533]
INFO:tensorflow:Evaluation [1071/1533]
INFO:tensorflow:Evaluation [1224/1533]
INFO:tensorflow:Evaluation [1377/1533]
INFO:tensorflow:Evaluation [1530/1533]
INFO:tensorflow:Evaluation [1533/1533]
INFO:tensorflow:Finished evaluation at 2019-09-24-07:55:57
INFO:tensorflow:Saving dict for global step 60